Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Model Development

In the introductory lab, we explored ways to perform anomaly detection.  We implemented our own solution to labeling data points that were several standard deviations from the mean of the data points.  Then we estimated linear and seasonal trends in the time series, and removed them to perform anomaly detection on the remaining time series.

Here, we are going to apply this approach to our telemetry data.  We will first develop a solution that processes all the data in batch, and then optimize the solution so that it detects anomalies online, as a new sensor measurement arrives.

In this lab, we will do the following:
- Use python port [pyculiarity](https://github.com/nicolasmiller/pyculiarity) of [Twitter's anomaly detection R library](https://github.com/twitter/AnomalyDetection)
- Apply this library to detect anomalies in our telemetry data
- Optimize our approach to perform online anomaly detection

## Prerequisites:

In order to run this lab, you may have to install these additional python packages (unless you already installed them):
- pyculiarity
- numpy

Do you remember from the previous lab how to do this?

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import os
from pyculiarity import detect_ts # python port of Twitter AD lib
import matplotlib.pyplot as plt # for plotting
import seaborn # for decent default settings for figures
import time # so we can time our operations

## Load data

As usual, we start by loading the data

In [ ]:
print("Reading data ... ", end="")
telemetry = pd.read_csv(os.path.join('..', 'data', 'telemetry.csv'))
print("Done.")

print("Parsing datetime...", end="")
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'], format="%m/%d/%Y %I:%M:%S %p")
print("Done.")

print("Reading data ... ", end="")
anoms_batch = pd.read_csv(os.path.join('..', 'data', 'anoms.csv'))
anoms_batch['datetime'] = pd.to_datetime(anoms_batch['datetime'], format="%Y-%m-%d %H:%M:%S")
print("Done.")

## Data Preparation

Let's pick a random sensor (e.g. volt) and machine id and see what we get.

In [ ]:
sensor = 'volt'
machine_id = 67

# Select the slice of the telemetry for this machine 
df_s = telemetry.loc[telemetry.loc[:, 'machineID'] == machine_id, ('datetime', sensor)]
df_s.columns = ['timestamp', 'value']

In [ ]:
# let's take a peak at the data
fig = plt.figure(figsize=(16,4))
plt.plot(df_s['timestamp'], df_s['value'])
df_s.describe()

In [ ]:
results = detect_ts(df_s) # we use the detect_ts function of pyculiarity
anoms = results['anoms']

plt.plot(df_s['timestamp'], df_s['value'])
plt.plot(anoms['timestamp'], anoms['anoms'], 'o')

Hmmm. Looks like we don't really get too many anomalies this way.  We could try to lower our thresholds. Should we try a lower threshold?

In [ ]:
alpha = 0.2 # The level of statistical significance with which to accept or reject anomalies. (default: 0.05)

results = detect_ts(df_s, alpha=alpha)
anoms = results['anoms']

plt.plot(df_s['timestamp'], df_s['value'])
plt.plot(anoms['timestamp'], anoms['anoms'], 'o')

That doesn't really seem too change to much. We could lower the threshold further, but maybe we need to take a completely different approach.

## Denoising data

The data look really noisy. Maybe what we need to do is to calculate running averages, because it is not individual measurements that are a problem, but a *collection* of measurements.

Let's start by aggregating sensor measurements over the last 12h.  Let's use exponential decay, so that more recent measurements inside the time window have more weight that those that are at the beginning of the time window.

In [ ]:
com = 6 # Specify decay in terms of center of mass (this approximates averageing over about twice as many hours)
rm = df_s['value'].ewm(com=com).mean()

plt.plot(df_s['timestamp'], df_s['value'])
plt.plot(df_s['timestamp'], rm)

Interesting, now there seem to be some more apparent anomalies.  Apparently, sometimes sensor readings are off for several hours in a row.  Let's try to run anomaly detection on the smoother time series.

In [ ]:
# we create a data frame with two columns (timestamp, value), which is what pyculiarity expects as input
df_smooth = pd.DataFrame(data={'timestamp': df_s['timestamp'], 'value': rm})

results = detect_ts(df_smooth)
anoms = results['anoms']

# we store the location of the last anomaly for later
last_anomaly = int(np.where(df_smooth['timestamp'] == results['anoms'].tail(1)['timestamp'].values[0])[0])

In [ ]:
plt.plot(df_smooth['timestamp'], df_smooth['value'])
plt.plot(anoms['timestamp'], anoms['anoms'], 'o', alpha=.2)
plt.xlabel("Dates")
plt.ylabel("Sensor Value")

### Hands-on lab

That appears to work, but we want our scoring service to work in real-time.  That means we shouldn't be calculating running averages over all the historic data whenever we receive just one new row of data.  Instead we want to use the last existing value of the running average, integrate it with the new data, to get the new running average.  Put another way, we want to calculate running averages *online*, rather than in *batch*.

### Calculating running averages online

The first step is to create a lightweight function that can calculate running avgs. 

[Welford's online](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Welford's_Online_algorithm) algorithm for calculating running variance includes a method to also calculate the mean.  This method is also described in Donald Knuth's Art of Computer Programming, Vol 2, page 232, 3rd edition.  

In [ ]:
import numpy as np

def run_avg(ts):
    rm_o = np.zeros_like(ts)
    rm_o[0] = -1 # enter your solution here

    for r in range(1, len(rm)):
        curr_com = float(min(com, r))
        rm_o[r] = -1 # enter your solution here
    return rm_o

In [ ]:
# If you have trouble coming up with the exact solution, please uncomment below
# %load "../solutions/running_avg.py"

 Let's confirm that your solution is correct. If your online version produces the same results as the batch version, you should just see an orange line in the below plot.  That is, a perfect blend of red and yellow. If you also see red and yellow, that means your online solution falls short of implmenting the batch version for calculating running averages.

In [ ]:
rm_o = run_avg(df_s['value'].values)

plt.plot(df_smooth['timestamp'], df_smooth['value'], color='yellow')
plt.plot(df_smooth['timestamp'], rm_o, alpha=.5, color='red')

You can also plot he difference between the two resulting time series.  Except for the couple of first values in the timeseries, this should all be zeros.

In [ ]:
plt.plot(df_smooth['timestamp'], df_smooth['value'] - rm_o)

### End of Lab

## Optimizing the window over which to perform Anomaly Detection

One remaining optimization is to only use the last couple of values of the running average for anomaly detection, instead of running it repeatedely on the entire timeseries of the running average.  There is no general answer for how many recent values have to be included to get robust anomaly detection, because the choice of the size of this sliding window depends on all sorts of factors (e.g. machine, sensor, size of anomaly).  

This means we have to develop an empirical approach to finding the right size for this sliding window.   

The first step is to define a function that performs anomaly detection using only those values that fall within a specified time window leading up to the current value.

The smaller the window, the faster the anomaly detection algorithm runs.  However, if we make the window too small, the algorithm will no longer be able to detect linear and seasonal trends, and will fail.

Below is a function that allows you to test different window sizes for anomaly detection.  It expects a data frame with two columns (timestamp, value), a specification of the window size, and the row index of an anomaly in the provided data frame.  The function returns whether it found an anomaly and how long it took to run (in seconds).

In [ ]:
def detect_ts_online(df_smooth, window_size, stop):
    is_anomaly = False
    run_time = 9999
    start_index = max(0, stop - window_size)
    df_win = df_smooth.iloc[start_index:stop, :]
    start_time = time.time()
    results = detect_ts(df_win, alpha=0.05, max_anoms=0.02, only_last=None, longterm=False, e_value=False, direction='both')
    run_time = time.time() - start_time
    if results['anoms'].shape[0] > 0:
        timestamp = df_win['timestamp'].tail(1).values[0]
        if timestamp == results['anoms'].tail(1)['timestamp'].values[0]:
            is_anomaly = True
    return is_anomaly, run_time

let's run this function to test whether we can recover the last anomaly we previously discovered in this time series using the batch method.


In [ ]:
is_anomaly, run_time = detect_ts_online(df_smooth, 24*7, last_anomaly) 

print('Success: %s, run_time: %ss' % (is_anomaly, round(run_time,2)))

### Hands-on lab

We need to systematically explore how different window sizes affect our ability to detect anomalies in the time series.  When we do this, we need to consider both our precision and recall in doing so.  Depending on the operational cost of false positives and negatives, we may have to use different F-measures.

In order to do this, we define a function that samples random rows and sensors from the data frame.  It checks whether the batch anomaly detection algorithm detected an anomaly at that time point, and tests whether the online anomaly detection algorithm comes to the same result. 

It records the number of correct and incorrect anomaly detection results, and calculate accuracy measures.

The script accepts two input arguments:
- window_size: how much historic data to retain
- com: Specify decay in terms of center of mass for running avg


We already provided the script for doing this below. But we would like you to discuss with your neighbors which of the following metrics you should use:
- precision
- recall
- f1-score
- f2-score (fbeta_score, w/ beta = 2)
- f0.5 score

You can read up on those in the sklearn documentation of [metrics](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html).

Think about the cost and savings of different settings for the size of the sliding window.  What if the cost of a false negative is high (e.g. USD 50000 for a broken machine), what if that cost is low (e.g. USD 10 for replacing a broken part and restarting the machine)?  What if a false positive is really expensive (e.g. USD 1000 for production loss while machine is being restarted)?

To conclude the lab, enter your choice at the bottom of the next script and run it.

### End of lab

In [ ]:
# %%writefile ../solutions/sample_run_solution.py

def sample_run(df, window_size = 500, com = 12):
    """
    This functions expects a dataframe df as mandatory argument.  
    The first column of the df should contain timestamps, the second machine IDs
    
    Keyword arguments:
    n_machines_test: the number of machines to include in the sample
    ts_per_machine: the number of timestamps to test for each machine
    window_size: the size of the window of data points that are used for anomaly detection
    
    """

    n_epochs = 10
    p_anoms = .5
    
    # create arrays that will hold the results of batch AD (y_true) and online AD (y_pred)
    y_true = []
    y_pred = []
    run_times = []
    
    # check which unique machines, sensors, and timestamps we have in the dataset
    machineIDs = df['machineID'].unique()
    sensors = df.columns[2:]
    timestamps = df['datetime'].unique()[window_size:]
    
    # sample n_machines_test random machines and sensors 
    random_machines = np.random.choice(machineIDs, n_epochs)
    random_sensors = np.random.choice(sensors, n_epochs)

    # we intialize an array with that will later hold a sample of timetamps
    random_timestamps = np.random.choice(timestamps, n_epochs)
    
    for i in range(0, n_epochs):
        # take a slice of the dataframe that only contains the measures of one random machine
        df_s = df[df['machineID'] == random_machines[i]]
        
        # smooth the values of one random sensor, using our run_avg function
        smooth_values = run_avg(df_s[random_sensors[i]].values, com)
        
        # create a data frame with two columns: timestamp, and smoothed values
        df_smooth = pd.DataFrame(data={'timestamp': df_s['datetime'].values, 'value': smooth_values})

        # load the results of batch AD for this machine and sensor
        anoms_s = anoms_batch[((anoms_batch['machineID'] == random_machines[i]) & (anoms_batch['errorID'] == random_sensors[i]))]
                
        # find the location of the t'th random timestamp in the data frame
        if np.random.random() < p_anoms:
            anoms_timestamps = anoms_s['datetime'].values
            np.random.shuffle(anoms_timestamps)
            counter = 0
            while anoms_timestamps[0] < timestamps[0]:
                if counter > 100:
                    return 0.0, 9999.0
                np.random.shuffle(anoms_timestamps)
                counter += 1
            random_timestamps[i] = anoms_timestamps[0]
            
        # select the test case
        test_case = df_smooth[df_smooth['timestamp'] == random_timestamps[i]]
        test_case_index = test_case.index.values[0]

        # check whether the batch AD found an anomaly at that time stamps and copy into y_true at idx
        y_true_i = random_timestamps[i] in anoms_s['datetime'].values

        # perform online AD, and write result to y_pred
        y_pred_i, run_times_i = detect_ts_online(df_smooth, window_size, test_case_index)
        
        y_true.append(y_true_i)
        y_pred.append(y_pred_i)
        run_times.append(run_times_i)
        
        score = <put your solution here>
        run_time = np.mean(run_times)
        
    return score, run_time
    

In [ ]:
# if you are running into issues, please use the following link
# %load ../solutions/sample_run_solution.py

Explore the parameter space:
- Different settings for the size of the sliding window (e.g. 50, 100, 5000, 1000).
- Different settigns for the decay in terms of center of mass for running avg (e.g. 4,6,8,12 24)

In [ ]:
from sklearn.metrics import fbeta_score

n_epochs = 10
window_size = 500
com = 12

fscore, run_time = sample_run(telemetry, window_size=window_size, n_epochs = n_epochs, com=com)
fscore

## Hyper Parameter Tuning with HyperDrive and BatchAI

If you are getting board of trying to find the right hyperparameters, you should stop searching immediately, because after the next lab, we will show you how to use BatchAI to do this tedious labor for you!

If you want to, you can already read up on HyperDrive:
- [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters)
- [sample notebook](https://github.com/Azure/MachineLearningNotebooks/tree/master/training/03.train-hyperparameter-tune-deploy-with-tensorflow)

## The end

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.